# TF-IDF

The idea is use the simple approch of tf-idf using panda and sklearn. If training is slow, launch a large instance in aws to run an extensive grid search.

## Quick look at the shape of the data

In [ ]:
import pandas as pd

data = pd.read_csv("./data/train.csv")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:
data.describe()

## Predictions only for toxic

In [ ]:
from sklearn.model_selection import StratifiedShuffleSplit

split = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=543553L)

In [ ]:
trainIndex, testIndex = list(split.split(data, data.toxic))[0]

In [ ]:
train, test = data.iloc[trainIndex], data.iloc[testIndex]

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer().fit(train.comment_text)

In [ ]:
trainFeatures = tfidf.transform(train.comment_text)

In [ ]:
from sklearn.linear_model import LogisticRegression

model = LogisticRegression().fit(trainFeatures, train.toxic)

In [ ]:
predictions = model.predict(tfidf.transform(test.comment_text))

In [ ]:
predictionProbabilities = model.predict_proba(tfidf.transform(test.comment_text))

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score, log_loss

print("Confusion matrix:")
print(confusion_matrix(test.toxic, predictions))
print("F1 score: {}".format(f1_score(test.toxic, predictions, pos_label=1.0)))
print("Logarithmic loss: {}".format(log_loss(test.toxic, predictionProbabilities)))